# Basic Usage
This notebook illustrates the basic usage to create incremental explanations with our incremental explainers.

### Import Modules
Imports all necessary modules at once.

In [ ]:
from river.metrics import CrossEntropy, Accuracy
from river.utils import Rolling
from river.ensemble import AdaptiveRandomForestClassifier

from river.datasets.synth import Agrawal

from increment_explain.explainer import IncrementalPFI
from increment_explain.explainer.sage import IncrementalSage, IntervalSage
from increment_explain.imputer import MarginalImputer
from increment_explain.storage import GeometricReservoirStorage
from increment_explain.utils.wrappers.river import RiverPredictionFunctionWrapper
from increment_explain.visualization import FeatureImportancePlotter

### Setup data stream

In [ ]:
stream = Agrawal(classification_function=1, seed=42)
feature_names = list([x_0 for x_0, _ in stream.take(1)][0].keys())
n_samples = 25_000

### Setup model and loss function

In [ ]:
model = AdaptiveRandomForestClassifier(n_models=15, max_depth=10, leaf_prediction='mc')
model_function = RiverPredictionFunctionWrapper(model.predict_proba_one)

loss_metric = CrossEntropy()
training_metric = Rolling(CrossEntropy(), window_size=1000)

### Setup Explainers
The explainer also require an imputer and a storage mechanism.

In [ ]:
storage = GeometricReservoirStorage(
    size=200,
    store_targets=False
)

imputer = MarginalImputer(
    model_function=model_function,
    storage_object=storage,
    sampling_strategy="joint"
)

incremental_sage = IncrementalSage(
    model_function=model_function,
    loss_function=loss_metric,
    imputer=imputer,
    storage=storage,
    feature_names=feature_names,
    smoothing_alpha=0.001,
    n_inner_samples=1
)

interval_sage = IntervalSage(
    model_function=model_function,
    loss_function=loss_metric,
    feature_names=feature_names,
    interval_length=2000,
    n_inner_samples=1
)

incremental_pfi = IncrementalPFI(
    model_function=model_function,
    loss_function=loss_metric,
    imputer=imputer,
    storage=storage,
    feature_names=feature_names,
    smoothing_alpha=0.001,
    n_inner_samples=1
)

sage_plotter = FeatureImportancePlotter(feature_names=feature_names)
pfi_plotter = FeatureImportancePlotter(feature_names=feature_names)

### Train and explain the models

In [ ]:
for (n, (x_i, y_i)) in enumerate(stream, start=1):
    # predicting
    y_i_pred = model.predict_proba_one(x_i)
    training_metric.update(y_true=y_i, y_pred=y_i_pred)

    # sage inc
    inc_fi_sage = incremental_sage.explain_one(x_i, y_i)
    int_fi_sage = interval_sage.explain_one(x_i, y_i)
    inc_fi_pfi = incremental_pfi.explain_one(x_i, y_i, update_storage=False)

    # visualize
    sage_plotter.update(inc_fi_sage, facet_name='inc-sage')
    sage_plotter.update(int_fi_sage, facet_name='int-sage')
    pfi_plotter.update(inc_fi_pfi, facet_name='inc-pfi')

    # learning
    model.learn_one(x_i, y_i)

    if n % 1000 == 0:
        print(f"{n}: perf                {training_metric.get()}\n"
              f"{n}: x_i                 {x_i}\n"
              f"{n}: inc-sage            {incremental_sage.importance_values}\n"
              f"{n}: int-sage            {interval_sage.importance_values}\n"
              f"{n}: pfi-sage            {incremental_pfi.importance_values}\n"
              f"{n}: diff                {incremental_sage.marginal_loss - incremental_sage.model_loss}\n"
              f"{n}: marginal-loss       {incremental_sage.marginal_loss}\n"
              f"{n}: model-loss          {incremental_sage.model_loss}\n"
              f"{n}: sum-sage            {sum(list(incremental_sage.importance_values.values()))}\n")

    if n >= n_samples:
        break

### Visualize the explanations

In [ ]:
# TODO add plotter